In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sys
sys.path.append("../")
import src.Resources as src

In [29]:
movies = pd.read_csv('../Data/Movies_clean.csv', index_col= 0)  
books = pd.read_csv('../Data/Books_clean.csv', index_col= 0)  

In [30]:
movies.columns = ["Title","Image","Genre","Description","Rating","Votes","Directors","Stars"]
books.columns = ["Author","Image","Description","Rating","Votes","Title","Genre"]

In [31]:
movies["Stars"].fillna("Nostars",inplace=True)
movies.drop_duplicates(inplace=True)
movies = movies.drop_duplicates(subset="Title")
movies = movies.reset_index()
movies = movies.drop("index",axis=1)

books["Description"].fillna("No description",inplace=True)
books.drop_duplicates(inplace=True)
books = books.drop_duplicates(subset="Title")
books = books.reset_index()
books = books.drop("index",axis=1)

In [33]:
director = movies["Directors"].apply(lambda x: x.split("|"))
m_genre = movies["Genre"].apply(lambda x: x.split("|"))
stars = movies["Stars"].apply(lambda x: x.split("|"))
b_genre = books["Genre"].apply(lambda x: x.split("|"))

In [34]:
n_directors = len(director.explode().unique())
movie_genres = len(m_genre.explode().unique())
movie_stars = len(stars.explode().unique())
book_genre = len(b_genre.explode().unique())
n_authors = books["Author"].nunique()
n_titles = books["Title"].nunique()
n_titles_m = movies["Title"].nunique()

In [35]:
Bignumbers = pd.DataFrame([n_authors], columns=["Book_Authors"])
Bignumbers["Movie_directors"] = n_directors
Bignumbers["Movie_stars"] = movie_stars
Bignumbers["Book_titles"] = n_titles
Bignumbers["Movie_titles"] = n_titles_m
Bignumbers["Book_genres"] = book_genre
Bignumbers["Movie_genres"] = movie_genres

In [36]:
Bignumbers

,Book_Authors,Movie_directors,Movie_stars,Book_titles,Movie_titles,Book_genres,Movie_genres
0,5219,38261,129170,12527,77900,414,28


In [37]:
movies = movies[movies["Votes"]>1000]
books = books[books["Votes"]>1000]

In [38]:
max_m =movies["Rating"].max()
min_m = movies["Rating"].min()
median_m = movies["Rating"].median()
mean_m = movies["Rating"].mean()

In [39]:
max_b = books["Rating"].max()
min_b = books["Rating"].min()
median_b = books["Rating"].median()
mean_b = books["Rating"].mean()

In [40]:
metric = ["Max","Min","Median","Mean","Max","Min","Median","Mean"]
rating = [max_m,min_m,median_m,mean_m,max_b,min_b,median_b,mean_b]
type = ["Movie","Movie","Movie","Movie","Book","Book","Book","Book"]


In [41]:
Data = pd.DataFrame({"Metric": metric, "Rating": rating, "Type": type})

In [15]:
fig = px.bar(Data, x="Metric",y="Rating",color="Type",barmode="group")
fig.show()

In [43]:
books['Genre'] = books['Genre'].apply(src.map_genre)

In [45]:
books.dropna(inplace=True)
movies.dropna(inplace=True)
books["Genre"] = books["Genre"].apply(lambda x: x.split("|"))
movies["Genre"] = movies["Genre"].apply(lambda x : x.split("|"))

In [46]:
genre_books = books["Genre"].explode().value_counts().reset_index()
genre_movies = movies["Genre"].explode().value_counts().reset_index()

In [47]:
genre_books["Type"]= "Book"
genre_movies["Type"] = "Movie"

In [48]:
genre_total = pd.concat([genre_books,genre_movies])

In [49]:

df = genre_total

fig = px.bar(
    genre_total,
    x = "index",
    y = "Genre",
    color = "Type",
    barmode="group",
    color_discrete_map = {"Book": "#44CF6C", "Movie": "#0E7C7B"})

fig.update_layout(
    plot_bgcolor = "#E1D5D7",
    paper_bgcolor = "#F7EBEC",
    font = dict(family = "Arial", color = "#00072D"))
    
fig.show()

In [22]:
#Title/Genre/Rating/Type

In [53]:
books_exploded = pd.DataFrame({"Title": books["Title"], "Rating": books["Rating"],"Genre": books["Genre"], "Type": "Books"})
movies_exploded = pd.DataFrame({"Title": movies["Title"], "Rating": movies["Rating"],"Genre": movies["Genre"], "Type": "Movies"})

In [60]:
books_exploded = books_exploded.explode("Genre").reset_index(drop=True)
movies_exploded = movies_exploded.explode("Genre").reset_index(drop=True)

In [62]:
movies_exploded.head(1)

,index,Title,Rating,Genre,Type
0,0,Dungeons & Dragons: Honor entre ladrones,7.6,Action,Movies


In [ ]:
def genre_graph(df,genre):